In [ ]:
#기사 목록

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from urllib.parse import quote
import pandas as pd
import time

# ✅ 검색어 목록
queries = [
    "박경리"
]

# ✅ 검색 기간
start_date = "2000-01-01"
end_date = "2024-08-31"

# ✅ 셀레니움 설정
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(), options=options)

# ✅ 결과 저장용 리스트 및 중복 URL 필터
results = []
seen_urls = set()

for query in queries:
    encoded_query = quote(query)
    page = 1
    print(f"[+] 검색어: {query}")

    while True:
        url = f"https://search.khan.co.kr/?q={encoded_query}&media=khan&page={page}&section=0&term=5&startDate={start_date}&endDate={end_date}&sort=1"
        driver.get(url)
        time.sleep(2)

        articles = driver.find_elements(By.CSS_SELECTOR, "ul.list > li > ul > li")
        if not articles:
            print(f"  - 페이지 {page}에 기사 없음, 중단")
            break

        new_articles_found = False

        for article in articles:
            # 제목과 링크
            try:
                a_tag = article.find_element(By.CSS_SELECTOR, "div > a")
                title = a_tag.text.strip()
                link = a_tag.get_attribute("href")
            except:
                continue

            if link in seen_urls:
                continue

            # 작성자 + 작성일
            writer = ""
            date = ""
            try:
                date_tag = article.find_element(By.CSS_SELECTOR, "p.etc.last")
                spans = date_tag.find_elements(By.TAG_NAME, "span")
                if len(spans) == 2:
                    writer = spans[0].text.strip()
                    date_raw = spans[1].text.strip()
                elif len(spans) == 1:
                    date_raw = spans[0].text.strip()
                else:
                    date_raw = ""
                date = date_raw.split()[0] if date_raw else ""
            except:
                pass

            # 카테고리
            try:
                category = article.find_element(By.CSS_SELECTOR, "p.category").text.strip()
            except:
                category = ""

            # 저장
            results.append({
                "media": '경향신문',
                "title": title,
                "date": date,
                "section": category,
                'article_type': '',
                "writer": writer,
                'page': '',
                "url": link,
                'content': ''  # 컨텐츠 정보 없음
            })
            seen_urls.add(link)
            new_articles_found = True

        if not new_articles_found:
            print(f"  - 페이지 {page}에 새로운 기사 없음, 중단")
            break

        page += 1
        print(f"  - 페이지 {page} 완료")

# 엑셀 파일 저장 함수
def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df.to_excel(filename, index=False)


# 결과를 엑셀로 저장
save_to_excel(results, f'1경향신문 {start_date}~{end_date}.xlsx')

# 드라이버 종료
driver.quit()

In [ ]:
# 본문 추출

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

# URL에서 본문과 부제를 스크랩하는 함수 정의
def get_content_and_subtitle_from_url(url):
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 경향신문 본문 추출
        article_body = soup.find('div', class_='art_body')

        # 부제 추출 (div.editor-subtitle)
        subtitle_tag = soup.find('div', class_='editor-subtitle')
        subtitle = ""
        if subtitle_tag:
            subtitle = subtitle_tag.get_text(separator='\n', strip=True)
            subtitle_tag.decompose()  # 부제를 본문에서 제거

        if article_body:
            # 제거할 태그 및 클래스를 리스트로 관리
            tags_to_remove = [
                ('div', 'art_photo_wrap'),   # 사진 캡션 부분
                ('div', 'inner.subscribeBtn.btm')  # 구독 관련 부분
            ]

            # 리스트 내의 태그와 클래스를 찾아 제거
            for tag, class_name in tags_to_remove:
                elements_to_remove = article_body.find_all(tag, class_=class_name)
                for element in elements_to_remove:
                    element.decompose()

            # 본문 텍스트 추출
            content = article_body.get_text(separator='\n', strip=True)
        else:
            content = ""
        return content, subtitle
    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return None, None
    
error_log = []
total_articles = len(df)    

# 각 행의 URL에서 본문과 부제 내용을 스크랩하여 content와 sub-title 컬럼에 저장합니다.
for index, row in df.iterrows():

    if pd.notna(row.get('content')) and row['content'].strip() != "":
        print(f"Skipping {index + 1}/{total_articles}: {row['url']} (already has content)")
        continue

    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content, subtitle = get_content_and_subtitle_from_url(row['url'])
    
    if content is None:
        error_log.append(row['url'])
        content = ""
    
    if content.startswith('='):
        content = "'" + content
        
    df.at[index, 'content'] = content
    df.at[index, 'sub-title'] = subtitle  # 새로운 부제 컬럼에 저장

    # 100개마다 파일 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)

print(f"본문 내용을 성공적으로 스크랩하여 저장했습니다: {output_path}")

# 에러 로그 파일 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")
    print(f"에러 로그가 저장되었습니다: {error_log_path}")



In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from urllib.parse import quote
import pandas as pd
import time

# ✅ 검색어 목록
queries = [
    "박경리"
]

# ✅ 검색 기간
start_date = "2000-01-01"
end_date = "2024-08-31"

# ✅ 셀레니움 설정
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(), options=options)

# ✅ 결과 저장용 리스트 및 중복 URL 필터
results = []
seen_urls = set()

for query in queries:
    encoded_query = quote(query)
    page = 1
    print(f"[+] 검색어: {query}")

    while True:
        url = f"https://search.khan.co.kr/?q={encoded_query}&media=khan&page={page}&section=0&term=5&startDate={start_date}&endDate={end_date}&sort=1"
        driver.get(url)
        time.sleep(2)

        articles = driver.find_elements(By.CSS_SELECTOR, "ul.list > li > ul > li")
        if not articles:
            print(f"  - 페이지 {page}에 기사 없음, 중단")
            break

        new_articles_found = False

        for article in articles:
            # 제목과 링크
            try:
                a_tag = article.find_element(By.CSS_SELECTOR, "div > a")
                title = a_tag.text.strip()
                link = a_tag.get_attribute("href")
            except:
                continue

            if link in seen_urls:
                continue

            # 작성자 + 작성일
            writer = ""
            date = ""
            try:
                date_tag = article.find_element(By.CSS_SELECTOR, "p.etc.last")
                spans = date_tag.find_elements(By.TAG_NAME, "span")
                if len(spans) == 2:
                    writer = spans[0].text.strip()
                    date_raw = spans[1].text.strip()
                elif len(spans) == 1:
                    date_raw = spans[0].text.strip()
                else:
                    date_raw = ""
                date = date_raw.split()[0] if date_raw else ""
            except:
                pass

            # 카테고리
            try:
                category = article.find_element(By.CSS_SELECTOR, "p.category").text.strip()
            except:
                category = ""

            # 저장
            results.append({
                "검색어": query,
                "제목": title,
                "URL": link,
                "작성자": writer,
                "작성일": date,
                "카테고리": category
            })
            seen_urls.add(link)
            new_articles_found = True

        if not new_articles_found:
            print(f"  - 페이지 {page}에 새로운 기사 없음, 중단")
            break

        page += 1
        print(f"  - 페이지 {page} 완료")

# ✅ 브라우저 종료
driver.quit()

# ✅ 엑셀 저장
df = pd.DataFrame(results)
df.to_excel("khan_articles_total.xlsx", index=False)
print(f"[✓] 총 {len(df)}개의 기사 저장 완료.")


[+] 검색어: 박경리
  - 페이지 2 완료
  - 페이지 3 완료
  - 페이지 4 완료
  - 페이지 5 완료
  - 페이지 6 완료
  - 페이지 7 완료
  - 페이지 8 완료
  - 페이지 9 완료
  - 페이지 10 완료
  - 페이지 11 완료
  - 페이지 12 완료
  - 페이지 13 완료
  - 페이지 14 완료
  - 페이지 15 완료
  - 페이지 16 완료
  - 페이지 17 완료
  - 페이지 18 완료
  - 페이지 19 완료
  - 페이지 20 완료
  - 페이지 21 완료
  - 페이지 22 완료
  - 페이지 23 완료
  - 페이지 24 완료
  - 페이지 25 완료
  - 페이지 26 완료
  - 페이지 27 완료
  - 페이지 28 완료
  - 페이지 29 완료
  - 페이지 30 완료
  - 페이지 31 완료
  - 페이지 32 완료
  - 페이지 33 완료
  - 페이지 34 완료
  - 페이지 35 완료
  - 페이지 36 완료
  - 페이지 37 완료
  - 페이지 38 완료
  - 페이지 39 완료
  - 페이지 40 완료
  - 페이지 41 완료
  - 페이지 42 완료
  - 페이지 43 완료
  - 페이지 44 완료
  - 페이지 45 완료
  - 페이지 46 완료
  - 페이지 47 완료
  - 페이지 48 완료
  - 페이지 49 완료
  - 페이지 50 완료
  - 페이지 50에 기사 없음, 중단
[✓] 총 488개의 기사 저장 완료.
